# Classification of Localizer Data

## Import necessary packages

In [6]:
%matplotlib inline

In [7]:
import glob
import os.path as op
import os as os
import nibabel as nib
import pandas as pd
import numpy as np

from nilearn.masking import compute_epi_mask

import matplotlib.pyplot as plt
import matplotlib as mpl

# Nilearn for neuro-imaging-specific machine learning
from nilearn.input_data import NiftiMasker
from nilearn import image

# Nibabel for general neuro-imaging tools
import nibabel

# Scikit-learn for machine learning
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import LeaveOneLabelOut, LeavePLabelOut, cross_val_score
from sklearn import preprocessing

# Plotting
import matplotlib.pyplot as plt
from nilearn import plotting
import seaborn as sns
sns.set(context="poster", style="ticks", font="Arial")


In [29]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, np.round(cm[i, j], decimals=2),
                 horizontalalignment="center", size='xx-large',
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
def plot_results(subj_info, cm_group, df_acc, df_auc=None, 
                 classes=['animal', 'face', 'fruitveg', 'tool', 'virtualtown']):
    
    if df_auc is not None:
        data = df_auc.merge(subj_info)
        print data.group.value_counts()/10
    
    plt.figure()
    plot_confusion_matrix(cm_group.mean(axis=0), classes=classes,
                          title='Mean confusion matrix')
    
    plt.figure()
    sns.factorplot(x='category', y='accuracy', hue='classifier', aspect=2,
                   units='subid', ci=68, data=df_acc, dodge=.1)
    
    plt.figure()
    sns.boxplot(x='category', y='accuracy', hue='classifier', data=df_acc)
    sns.stripplot(x='category', y='accuracy', hue='classifier', jitter=True, 
                  data=df_acc)
    
    if df_auc is not None:
        sns.factorplot(x='category', y='auc', hue='group', aspect=2,
                       units='subid', ci=68, data=data, dodge=.1)

#### Set up some colors for the plots

In [8]:
palette = {'logreg': 'mediumseagreen',
           'chance': 'darkgray',
           'f1 score': 'teal'}

### Define some functions for classification

In [9]:
# While debugging:
%load_ext autoreload
%aimport ap_classify

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from ap_classify import *

## Set up directory & file information

In [11]:
smoothing = 'unsmoothed'
regspace = 'epi'
design = 'localizer_cond_mvpa.csv' # onset file in lyman-style
# design = 'localizer_subcat.csv' # onset file in lyman-style
smoothing_fwhm = 0
standardize = True
tr = float(2) # in seconds
tr_shift = 4.5 # seconds to shift forward by
ts_type = 'raw' # raw or residual
run_list = [7, 8]

basedir = '/Volumes/group/awagner/sgagnon/AP'
analydir = op.join(basedir, 'analysis/mvpa_raw')
subjfile = op.join(analydir, 'notebooks/subj_info.csv')
subj_info = pd.read_csv(subjfile)

# Filepath templates
if ts_type == 'raw':
    tsfilename = 'timeseries_xfm.nii.gz'
elif ts_type == 'residual':
    tsfilename = 'res4d_xfm.nii.gz'
tsfile = op.join(analydir, "{subid}", 'reg', regspace, 
                 smoothing, "run_{run_id}", tsfilename)
func_maskfile = op.join(analydir, "{subid}", 'reg', regspace, 
                        smoothing, "run_{run_id}", 'functional_mask_xfm.nii.gz')
maskfile = op.join(basedir, 'data', "{subid}", 'masks', 
                   "{mask_name}.nii.gz")
meanfile = op.join(analydir, "{subid}", 'preproc',
                   "run_{run_id}", 'mean_func.nii.gz')
onsetfile = op.join(basedir, 'data', "{subid}", 'design', design)

# Output templates
outnifti = op.join(analydir, "{subid}", 'importance_maps')

artifacts = op.join(analydir, '{subid}', 'preproc', 'run_{run}', 'artifacts.csv')

# Combine paths into dictionary (facilitate passing i/o of funcs)
paths = dict(tsfile=tsfile, func_maskfile=func_maskfile, 
             maskfile=maskfile, meanfile=meanfile, 
             onsetfile=onsetfile, outnifti=outnifti, 
             analydir=analydir, artifacts=artifacts)

We create anatomical masks in native space from a cortical parcellation of the high-resolution T1
image obtained for each participant using FreeSurfer and the resulting **bilateral
inferior temporal cortex**, **fusiform gyrus**, and **parahippocampal gyrus** were combined to serve
as the mask for MVPA classification (as in Zeithamova et al., 2012; *Neuron*).

## Run Classification (training/testing on 3 categories)

In [13]:
subj_info.group.value_counts()

stress     22
control    22
Name: group, dtype: int64

### Run localizer CV:

In [ ]:
# Initialize some dataframes for storage
df = pd.DataFrame(columns=['subid', 'mask_name', 'category', 'type', 'mean', 'sd'])
df_acc = pd.DataFrame(columns=['subid', 'mask_name', 'category', 'classifier', 'accuracy', 'count'])
df_proba = pd.DataFrame(columns=['subid', 'mask_name', 'true_category', 'guess_category', 'classifier', 'probability'])
df_auc = pd.DataFrame()

n_permutations=0 # or if want to run permutation test, e.g., 100
if n_permutations > 0:
    iter_list = list(np.arange(1, n_permutations + 1))
    iter_list.insert(0, 'subid')
    d_permute = pd.DataFrame(columns=iter_list)

mask_type = 'mask' # functional mask, or anatomical mask defined w/mask_name
mask_name = 'bilat-parahipp_fusi_inftemp_nohipp'
# mask_name = 'bilat-fusi_inftemp_nohipp' # excluding parahipp to see if just that
# mask_name = 'lh-inferiorparietal'
cond_list = ['face', 'object', 'place']
multi_class = 'ovr'
pca_n = None #or None
univariate_fsl_k=None # 1000 #or None

# mask_type = 'func' # functional mask, or anatomical mask defined w/mask_name
# mask_name = 'wholebrain'
# cond_list = ['face', 'object', 'place']

# Confusion matrix
cm_group = np.zeros((1,len(cond_list), len(cond_list)))

# Iterate through subjects
for subid in subj_info.subid:
    print subid
    
    onsetfile = paths['onsetfile']
    # Get subj-specific data
    X, run_labels, ev_labels, ev_trs, ev_onsets, func_masker = get_subj_data(subid, onsetfile, cond_list, paths, mask_type, mask_name,
                                                                             smoothing_fwhm, standardize, tr, tr_shift, run_list, 
                                                                             shift_rest=True, filter_artifacts=True)
    
    cv = LeaveOneLabelOut(run_labels)
    plot_validation_curve(X, ev_labels, cv)
    
    # Classification
    if n_permutations > 0:
        df, d_permute = calc_scores(df, subid, mask_name, X, ev_labels, run_labels, 
                                    n_permutations=n_permutations, d_permute=d_permute, 
                                    plot_permutation=True, multi_class=multi_class)
    else:
        df = calc_scores(df, subid, mask_name, X, ev_labels, run_labels, multi_class=multi_class)
    
    if multi_class == 'MLP':
        df_acc, df_proba, cm_group = calc_acc_proba(df_acc, df_proba, subid, mask_name, X, ev_labels, 
                                                    run_labels, cv,conf_mat=True, multi_class=multi_class, 
                                                    cm_group=cm_group)
    else:
        df_acc, df_proba, df_auc, cm_group = calc_acc_proba(df_acc, df_proba, subid, mask_name, X, ev_labels, 
                                                            run_labels, cv,conf_mat=True, multi_class=multi_class, 
                                                            univariate_fsel_k=univariate_fsl_k, undersampling=True,
                                                            pca_n=pca_n,
                                                            cm_group=cm_group,
                                                            compute_AUC=True, df_auc=df_auc,
                                                            repeated_ttest_fsel=None)
    
#     # Create coef maps by training on all data, save niis and pngs to dir
#     create_coef_maps(subid, X, ev_labels, func_masker, mask_name, paths, calc_A=False)

#### Various possible outputs

In [ ]:
df.to_csv('output_ap/localizer_scores_filterart_raw_scalewithinrun.csv')

In [ ]:
df_acc.to_csv('output_ap/localizer_accuracy_raw.csv', index=False)
df_proba.to_csv('output_ap/localizer_proba_raw.csv', index=False)

In [63]:
# VTC - no hipp
df_auc.to_csv('output_ap/localizer_vtcnohipp_auc_filterart_raw_scalewithinrun.csv')
df_acc.to_csv('output_ap/localizer_vtcnohipp_accuracy_filterart_raw_scalewithinrun.csv')
df_proba.to_csv('output_ap/localizer_vtcnohipp_proba_filterart_raw_scalewithinrun.csv')
df.to_csv('output_ap/localizer_vtcnohipp_df_filterart_raw_scalewithinrun.csv')
d_permute.to_csv('output_ap/localizer_vtcnohipp_permute_filterart_raw_scalewithinrun.csv')

In [49]:
# Whole hippocampus
df_auc.to_csv('output_ap/localizer_bilat-hipp_auc_filterart_raw_scalewithinrun.csv')
df_acc.to_csv('output_ap/localizer_bilat-hipp_accuracy_filterart_raw_scalewithinrun.csv')
df_proba.to_csv('output_ap/localizer_bilat-hipp_proba_filterart_raw_scalewithinrun.csv')
df.to_csv('output_ap/localizer_bilat-hipp_df_filterart_raw_scalewithinrun.csv')
d_permute.to_csv('output_ap/localizer_bilat-hipp_permute_filterart_raw_scalewithinrun.csv')

In [90]:
# inf parietal
df_auc.to_csv('output_ap/localizer_inferiorparietal_auc_filterart_raw_scalewithinrun.csv')
df_acc.to_csv('output_ap/localizer_inferiorparietal_accuracy_filterart_raw_scalewithinrun.csv')
df_proba.to_csv('output_ap/localizer_inferiorparietal_proba_filterart_raw_scalewithinrun.csv')
df.to_csv('output_ap/localizer_inferiorparietal_df_filterart_raw_scalewithinrun.csv')
d_permute.to_csv('output_ap/localizer_inferiorparietal_permute_filterart_raw_scalewithinrun.csv')